In [60]:
import pandas as pd
import re

# Korak 1: Branje HTML tabele neposredno iz URL-ja
url = 'https://kt.ijs.si/~ljupco/lectures/papvp-2425/podatkovni-viri/List%20of%20actors%20with%20Academy%20Award%20nominations%20-%20Wikipedia.html'
tables = pd.read_html(url, header=0)  # Preberemo tabelo in uporabimo prvo vrstico kot glavo

# Predpostavimo, da je druga tabela tista, ki nas zanima
data = tables[1]

# Preverimo, kakšna imena stolpcev imamo
print("Imena stolpcev v tabeli:")
print(data.columns)

# Uporabimo pravilna imena stolpcev, kot jih prebere pandas
actor_column = data.columns[0]  # Prvi stolpec (igralci)
born_column = data.columns[2]  # Stolpec z letom rojstva
died_column = data.columns[3]  # Stolpec z letom smrti
nominations_column = data.columns[5]  # Stolpec z nominacijami
wins_column = data.columns[6]  # Stolpec z zmagami
lead_supporting_column = data.columns[7]  # Stolpec z razdelitvijo vlog
film_column = data.columns[8]  # Stolpec s filmi

clean_data = []

# Funkcija za čiščenje številskih vrednosti (odstrani opombe in preveri veljavnost)
def clean_numeric(value):
    if pd.isnull(value):
        return 0
    value = str(value)
    value = re.sub(r'\[.*?\]', '', value)  # Odstranimo vse v oglatih oklepajih
    value = value.strip()  # Odstranimo morebitne presledke
    try:
        return int(value) if value.isdigit() else 0
    except ValueError:
        return 0

# Korak 2: Obdelava podatkov iz stolpcev
for _, row in data.iterrows():
    # Pridobimo podatke iz vsake vrstice
    actor = row[actor_column]
    birth_year = row[born_column]
    death_year = row[died_column] if row[died_column] != "~" else None
    nominations = clean_numeric(row[nominations_column])  # Očistimo nominacije
    wins = clean_numeric(row[wins_column])  # Očistimo zmage
    lead_supporting_details = row[lead_supporting_column]
    film = row[film_column]

    # Vedno razdelimo nominacije na podlagi Lead and Supporting details
    details_split = lead_supporting_details.split(':')
    main_noms = (
        int(details_split[0].strip('L')) if len(details_split) > 0 and details_split[0].strip('L').isdigit() else 0
    )
    supp_noms = (
        int(details_split[1].strip('S')) if len(details_split) > 1 and details_split[1].strip('S').isdigit() else 0
    )

    # Če nominacije ne ustrezajo razdelitvi, jih ročno razdelimo
    if main_noms + supp_noms != nominations:
        main_noms = nominations - supp_noms
        supp_noms = nominations - main_noms

    # Pravilna obdelava zmag
    main_wins = min(wins, main_noms)  # Zmage pri glavnih vlogah
    supp_wins = wins - main_wins  # Preostale zmage so pri stranskih

    # Pravilna razporeditev nominacij in zmag
    for main in ['Glavna', 'Stranska']:
        is_main = (main == 'Glavna')
        for win_status in [False, True]:  # Obrnemo vrstni red, da je najprej False, nato True
            clean_data.append({
                'Ime': actor,
                'Leto_rojstva': birth_year,
                'Leto_smrti': death_year,
                'Nominacija': main,
                'Oskar': win_status,
                'Število': (main_wins if is_main and win_status else
                            (main_noms - main_wins) if is_main and not win_status else
                            supp_wins if not is_main and win_status else
                            (supp_noms - supp_wins) if not is_main and not win_status else 0),
                'Prv_film': film
            })

# Korak 3: Ustvarjanje tabele
clean_df = pd.DataFrame(clean_data)

# Korak 4: Shranjevanje kot CSV
clean_df.to_csv('actors_award_nominations_cleaned.csv', index=False)
print("Tabela je bila shranjena kot 'actors_award_nominations_cleaned.csv'.")


Imena stolpcev v tabeli:
Index(['Actor', 'Unnamed: 1', 'Born', 'Died', 'Age', 'Nomina- tions', 'Wins',
       'Lead and sup- porting details',
       'First winning film role or first nomination (also see list of all nominated roles)',
       'First year', 'Last year'],
      dtype='object')
Tabela je bila shranjena kot 'actors_award_nominations_cleaned.csv'.
